In [144]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [145]:
!pip install openpyxl
df=pd.read_excel('flight_price.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [147]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [148]:
#Removing Duplicates
df.duplicated().sum()

np.int64(220)

In [149]:
df.drop_duplicates(inplace=True)

In [150]:
#Feature Engineering
df['date']=df['Date_of_Journey'].str.split('/').str[0]
df['month']=df['Date_of_Journey'].str.split('/').str[1]
df['year']=df['Date_of_Journey'].str.split('/').str[2]


In [151]:
df.drop('Date_of_Journey',axis=1,inplace=True)
df['date']=df['date'].astype(int)
df['month']=df['month'].astype(int)
df['year']=df['year'].astype(int)

In [152]:
df['Arrival_Time']=df['Arrival_Time'].apply(lambda x: x.split(' ')[0])

In [153]:
df['Arrival_hour']=df['Arrival_Time'].str.split(':').str[0].astype(int)
df['Arrival_minute']=df['Arrival_Time'].str.split(':').str[1].astype(int)

In [154]:
df.drop('Arrival_Time',axis=1,inplace=True)

In [155]:
df['Departure_hour']=df['Dep_Time'].str.split(':').str[0].astype(int)
df['Departure_minute']=df['Dep_Time'].str.split(':').str[1].astype(int)
df.drop('Dep_Time',axis=1,inplace=True)

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10463 entries, 0 to 10682
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           10463 non-null  object
 1   Source            10463 non-null  object
 2   Destination       10463 non-null  object
 3   Route             10462 non-null  object
 4   Duration          10463 non-null  object
 5   Total_Stops       10462 non-null  object
 6   Additional_Info   10463 non-null  object
 7   Price             10463 non-null  int64 
 8   date              10463 non-null  int64 
 9   month             10463 non-null  int64 
 10  year              10463 non-null  int64 
 11  Arrival_hour      10463 non-null  int64 
 12  Arrival_minute    10463 non-null  int64 
 13  Departure_hour    10463 non-null  int64 
 14  Departure_minute  10463 non-null  int64 
dtypes: int64(8), object(7)
memory usage: 1.3+ MB


In [157]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [158]:
df['Total_Stops'].mode()[0]

'1 stop'

In [159]:
df['Total_Stops']=df['Total_Stops'].map({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4,np.nan:1})

In [160]:
df.drop('Route',axis=1,inplace=True)
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute
0,IndiGo,Banglore,New Delhi,2h 50m,0,No info,3897,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2,No info,7662,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2,No info,13882,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1,No info,6218,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1,No info,13302,1,3,2019,21,35,16,50


In [161]:
# Extract the hour part
df['Duration_hour'] = df['Duration'].str.split(' ').str[0].str.split('h').str[0]

# Extract the minute part, making sure to call .strip() properly
df['Duration_minute'] = df['Duration'].str.split(' ').str[1].str.strip().str.split('m').str[0]


In [163]:
df['Duration_minute']=df['Duration_minute'].fillna('0')
df.drop('Duration',axis=1,inplace=True)
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,Duration_hour,Duration_minute
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,1,10,22,20,2,50
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,2019,13,15,5,50,7,25
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,6,2019,4,25,9,25,19,0
3,IndiGo,Kolkata,Banglore,1,No info,6218,12,5,2019,23,30,18,5,5,25
4,IndiGo,Banglore,New Delhi,1,No info,13302,1,3,2019,21,35,16,50,4,45


In [165]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
cat_features=['Airline','Source','Destination']
ohe_df=pd.DataFrame(ohe.fit_transform(df[cat_features]).toarray(),columns=ohe.get_feature_names_out(cat_features))
df=pd.concat([df,ohe_df],axis=1)
df.drop(cat_features,axis=1,inplace=True)
df.head()

,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.0,No info,3897.0,24.0,3.0,2019.0,1.0,10.0,22.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.0,No info,7662.0,1.0,5.0,2019.0,13.0,15.0,5.0,50.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.0,No info,13882.0,9.0,6.0,2019.0,4.0,25.0,9.0,25.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,No info,6218.0,12.0,5.0,2019.0,23.0,30.0,18.0,5.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,No info,13302.0,1.0,3.0,2019.0,21.0,35.0,16.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
